# EJERCICIO: COLOR
### Construye un clasificador de objetos en base a la similitud de los histogramas de color del ROI (de los 3 canales por separado)

El ejercicio se basa en diseñar un programa que, a partir de una imagen RGB ```img``` y unos ROIs seleccionados como modelos, determine cuál de ellos se aproxima más a una subimagen ```piece``` de ```img```. Para ello, se calcula los histogramas RGB (uno por cada canal) de cada ROI y se determina cuál de ellos presenta un menor **MSE** con respecto al histograma RGB de ```piece```.

Comenzamos con la importación de las librerías necesarias, la declaración de ```region``` para representar los ROIs en imagen y la creación de las ventanas:

- ```image``` se trata de la ventana donde se dibujará la imagen original ```img```
- ```models``` es la ventana para visualizar los modelos
- ```detected``` será donde mostremos el modelo que más se aproxima a ```piece```

In [14]:
import argparse
import cv2               as     cv
import numpy             as     np
import matplotlib.pyplot as plt
from   umucv.util        import ROI
from   collections       import deque
from   sklearn.metrics   import mean_squared_error

cv.namedWindow("image")
cv.moveWindow("image", 0, 0)

cv.namedWindow("models")
cv.moveWindow("models", 0, 530)

cv.namedWindow("detected")
cv.moveWindow("detected", 1030,0)

region = ROI("image")

Se definen, además, 3 funciones auxiliares:

- ```readrgb()``` recibe como parámetro el path de una imagen y la devuelve en formato RGB con un tamaño ajustado
- ```calcNormHistRGB()``` recibe una imagen RGB y devuelve el histograma de pixels normalizado de cada canal a lo largo de los 256 valores
- ```imAdjust()``` recibe como parámetro una imagen y le aplica un resize para que todos los ROIs tengan el mismo tamaño a la hora de visualizarlos.

In [15]:
def readrgb(file):
    return cv.cvtColor( cv.resize(cv.imread(file), (1000,500)), cv.COLOR_BGR2RGB)

def calcNormHistRGB(img):
    '''Dada una imagen RGB devuelve las frecuencias normalizadas de cada valor de pixel para cada canal de color '''
    hR,_ = np.histogram(img[:,:,0], np.arange(0, 256)) 
    hG,_ = np.histogram(img[:,:,1], np.arange(0, 256))
    hB,_ = np.histogram(img[:,:,2], np.arange(0, 256))
    return hR/np.sum(hR), hG/np.sum(hG), hB/np.sum(hB)

def adjustROI(im):
    return cv.resize(im, (333,300))

Estas serán las variables necesarias para la ejecución del programa:

- ```img``` será la imagen original que mostremos por pantalla.
- ```original_img``` se tratará de una copia de ```img```. Dado que será en ```img``` donde se dibujen los ROIs (un rectángulo amarillo), no sería correcto extraer ese mismo ROI como modelo debido a que las líneas del ROI interferirían en el resultado de los histogramas. Por ello, se utilizará ```original_img``` para extraer los valores RGB intrínsecos del ROI capturado como modelo.
- ```models``` y ```hisograms``` serán colas de longitud ```max``` donde almacenar los modelos y sus histogramas RGB (tecla 'C').
- ```detected``` será una lista de un único elemento: el modelo detectado que más se parezca a una subimagen seleccionada por el usuario (tecla 'SPACE').
- ```prevDims``` son las dimensiones del ROI en una iteración previa. De esta forma evitamos leer y dibujar el ROI más veces de las necesarias.
- ```cont``` es sencillamente un booleano para el control del bucle (el programa termina con la tecla 'ESC').

In [16]:
MAX = 3

img = readrgb("../images/falta2.jpg")
original_img = img.copy()
histograms = deque(maxlen = 3) 
models = deque(maxlen = 3)
detected = deque(maxlen = 1)
prevDims = []
cont = True

El procesamiento de la imagen se realiza en un bucle que continuará ejecutándose hasta que salgamos del programa (tecla 'ESC'). 

La primera comprobación que realizamos es la comparación entre el ROI anterior, cuyas dimensiones estarían almacenadas en ```prevDim```, y el actual. Solo merece la pena leer de nuevo la imagen y dibujar el ROI si este es uno distinto. Cabe mencionar que, en caso de de dibujar un ROI, es necesario hacerlo sobre una imagen que no disponga de otros ROIs, de lo contrario acabaríamos teniendo una mezcla de estos en una misma imagen. Por ello, si se está seleccionando un nuevo ROI, almacenamos una copia de ```original_img``` en ```img``` y se dibujará ahí.

```python
while cont:
    if region.roi:
        [x1,y1,x2,y2] = region.roi
        
        if (prevDims != [x1,y1,x2,y2]):
            prevDims = [x1,y1,x2,y2]
            img = original_img.copy()
            cv.rectangle(img, (x1,y1), (x2,y2), color=(0,255,255), thickness=2)
    
    #...
```

Lo siguiente a realizar es la obtención de la tecla pulsada (en un lapso de 1 ms). Si se capturó un ROI (aprovechamos ```prevDims``` para cerciorarnos de que existe un ROI), reajustamos su tamaño y añadimos toda la información a las listas donde proceda.

```python
    #...
    
    if key == ord('c') or key == ord('C') and prevDims:
        model = adjustROI(original_img[y1:y2+1, x1:x2+1])
        models.append(model)
        histograms.append((calcNormHistRGB(model)))
    
    #...
```

El siguiente paso es comprobar si se seleccionó una subimagen para poder detectar a qué modelo se acerca más. Para ello, reajustamos esta subimagen (todos los ROIs acaban teniendo el mismo tamaño), obtenemos los histogramas de cada canal RGB de la subimagen y calculamos el MSE que existe entre estos y los de cada modelo. Nos quedamos con el modelo que proporcione el MSE mínimo.

```python
    #...
    
    if key == 32 and models:
        """ piece = adjustROI(original_img[y1:y2+1, x1:x2+1])
        R,G,B = calcNormHistRGB(piece)
        rgbConcat = np.concatenate((R,G,B))

        mseList = []
        it = 1
        for hR,hG,hB in histograms:
            histContat = np.concatenate((hR,hG,hB))
            mse = mean_squared_error(rgbConcat, histContat)
            mseList.append(mse)
            print("MSE con el modelo " + str(it) +" = " + str(mse))
            it += 1 """
        
        # El codigo de arriba y el de abajo producen el mismo resultado
        mseList = [mean_squared_error(np.concatenate((calcNormHistRGB(original_img[y1:y2+1, x1:x2+1]))), np.concatenate((hR,hG,hB))) for (hR,hG,hB) in histograms]

        detected.append(models[np.argmin(mseList)])
        
        #...
```

Para terminar, se comprueba si la tecla que se pulsó fue para terminar el programa ('ESC'), se dibuja ```img``` con el posible ROI seleccionado, los modelos y el detectado para la clasificación (si procediera). 

````python
        #...
    
        if key == 27: 
            cont = False

        if (models):
            cv.imshow("models",np.hstack(models))

        cv.imshow("image",img)

        if (detected):
            cv.imshow("detected",detected[0])

cv.destroyAllWindows()
                
        # ...
````

Abajo queda listado todo este código listo para su ejecución.

In [17]:
while cont:
    if region.roi:
        [x1,y1,x2,y2] = region.roi
        # prevDims son las dimensiones del ROI en iteracion previa, de esta forma nos evitamos leer y dibujar el ROI mas veces de las necesarias
        if (prevDims != [x1,y1,x2,y2]):
            prevDims = [x1,y1,x2,y2]
            img = original_img.copy()
            cv.rectangle(img, (x1,y1), (x2,y2), color=(0,255,255), thickness=2)

    key = cv.waitKey(1)

    # Capturar el ROI = guardar el modelo
    if key == ord('c') or key == ord('C') and prevDims:
        # Modelos con tamaño ajustado para mostrarlos en pantalla
        model = adjustROI(original_img[y1:y2+1, x1:x2+1])
        models.append(model)
        # En la lista de histogramas guardamos los histogramas de cada canal de una imagen como una tupla (R,G,B)
        histograms.append((calcNormHistRGB(model)))


    # Seleccionar el modelo mas parecido al ROI indicado
    # Tecla espacio
    if key == 32 and models:
        """ piece = adjustROI(original_img[y1:y2+1, x1:x2+1])
        R,G,B = calcNormHistRGB(piece)
        rgbConcat = np.concatenate((R,G,B))

        mseList = []
        it = 1
        for hR,hG,hB in histograms:
            histContat = np.concatenate((hR,hG,hB))
            mse = mean_squared_error(rgbConcat, histContat)
            mseList.append(mse)
            print("MSE con el modelo " + str(it) +" = " + str(mse))
            it += 1 """

        mseList = [mean_squared_error(np.concatenate((calcNormHistRGB(original_img[y1:y2+1, x1:x2+1]))), np.concatenate((hR,hG,hB))) for (hR,hG,hB) in histograms]

        detected.append(models[np.argmin(mseList)])


    # Tecla ESC
    # Salir
    if key == 27: 
        cont = False

    if (models):
        cv.imshow("models",np.hstack(models))

    cv.imshow("image",img)

    if (detected):
        cv.imshow("detected",detected[0])

cv.destroyAllWindows()